In [1]:
# import necessariy libraries
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray

import numpy as np
from numpy import asarray
from matplotlib import pyplot
import os

## 1.2 Image Preprocessing

In [2]:
def data_loader ():
    # define the required paths
    base_path = 'textures/'
    train_test_path = ['training/' ,'testing/']
    folders_names = ['canvas1/', 'cushion1/', 'linsseeds1/', 'sand1/', 'seat2/', 'stone1/']

    train_canvas1, train_cushion1, train_linsseeds1, train_sand1, train_seat2, train_stone1 = [], [], [], [], [], []
    test_canvas1, test_cushion1, test_linsseeds1, test_sand1, test_seat2, test_stone1 = [], [], [], [], [], []
    images_list = [[train_canvas1, train_cushion1, train_linsseeds1, train_sand1, train_seat2, train_stone1],
                   [test_canvas1, test_cushion1, test_linsseeds1, test_sand1, test_seat2, test_stone1]]
    for i in range (0,2):

        # extract a list of file names in each directory
        folders_list = os.listdir( base_path+train_test_path[i] )
        print('Numebr of folders in',train_test_path[i],': ' ,len(folders_list))

        for j, item in enumerate(folders_list):
            files_list = os.listdir( base_path+train_test_path[i]+item+'/' )
            for k,file in enumerate(files_list):
                image = imread(base_path+train_test_path[i]+item+'/'+file)

                resized = resize(image, (32, 32), preserve_range=True) 
                gray = rgb2gray(resized)
                images_list[i][j].append(np.array(gray))
    #             pyplot.imshow(image)
    #             pyplot.show()
            images_list[i][j] = np.array(images_list[i][j])
        images_list[i] = np.array(images_list[i])

    train_canvas1, train_cushion1, train_linsseeds1, train_sand1, train_seat2, train_stone1 = images_list[0][0], images_list[0][1], images_list[0][2], images_list[0][3], images_list[0][4], images_list[0][5]
    test_canvas1, test_cushion1, test_linsseeds1, test_sand1, test_seat2, test_stone1 = images_list[1][0], images_list[1][1], images_list[1][2], images_list[1][3], images_list[1][4], images_list[1][5]
    # Create train set
    X_train = np.concatenate([train_canvas1, train_cushion1, train_linsseeds1, train_sand1, train_seat2, train_stone1])
    # Create label for train set
    y_train = np.zeros (X_train.shape[0])
    y_train[0:train_canvas1.shape[0]] = 0
    y_train[train_canvas1.shape[0]:train_canvas1.shape[0]+train_cushion1.shape[0]] = 1
    y_train[train_canvas1.shape[0]+train_cushion1.shape[0]:train_canvas1.shape[0]+train_cushion1.shape[0]+train_linsseeds1.shape[0]] = 2
    y_train[train_canvas1.shape[0]+train_cushion1.shape[0]+train_linsseeds1.shape[0]:train_canvas1.shape[0]+train_cushion1.shape[0]+train_linsseeds1.shape[0]+train_sand1.shape[0]] = 3
    y_train[train_canvas1.shape[0]+train_cushion1.shape[0]+train_linsseeds1.shape[0]+train_sand1.shape[0]:train_canvas1.shape[0]+train_cushion1.shape[0]+train_linsseeds1.shape[0]+train_sand1.shape[0]+train_seat2.shape[0]] = 4
    y_train[train_canvas1.shape[0]+train_cushion1.shape[0]+train_linsseeds1.shape[0]+train_sand1.shape[0]+train_seat2.shape[0]:train_canvas1.shape[0]+train_cushion1.shape[0]+train_linsseeds1.shape[0]+train_sand1.shape[0]+train_seat2.shape[0]+train_stone1.shape[0]] = 5
    
    
    # Creat test set
    X_test = np.concatenate([test_canvas1, test_cushion1, test_linsseeds1, test_sand1, test_seat2, test_stone1])
    # Create label for test set
    y_test = np.zeros (X_test.shape[0])
    y_test[0:test_canvas1.shape[0]] = 0
    y_test[test_canvas1.shape[0]:test_canvas1.shape[0]+test_cushion1.shape[0]] = 1
    y_test[test_canvas1.shape[0]+test_cushion1.shape[0]:test_canvas1.shape[0]+test_cushion1.shape[0]+test_linsseeds1.shape[0]] = 2
    y_test[test_canvas1.shape[0]+test_cushion1.shape[0]+test_linsseeds1.shape[0]:test_canvas1.shape[0]+test_cushion1.shape[0]+test_linsseeds1.shape[0]+test_sand1.shape[0]] = 3
    y_test[test_canvas1.shape[0]+test_cushion1.shape[0]+test_linsseeds1.shape[0]+test_sand1.shape[0]:test_canvas1.shape[0]+test_cushion1.shape[0]+test_linsseeds1.shape[0]+test_sand1.shape[0]+test_seat2.shape[0]] = 4
    y_test[test_canvas1.shape[0]+test_cushion1.shape[0]+test_linsseeds1.shape[0]+test_sand1.shape[0]+test_seat2.shape[0]:test_canvas1.shape[0]+test_cushion1.shape[0]+test_linsseeds1.shape[0]+test_sand1.shape[0]+test_seat2.shape[0]+test_stone1.shape[0]] = 5
    
    
    return X_train, X_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = data_loader()
X_train.shape , X_test.shape, y_train.shape, y_test.shape

Numebr of folders in training/ :  6


<ipython-input-2-70e6c94766ad>:23: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  gray = rgb2gray(resized)


Numebr of folders in testing/ :  6


((180, 32, 32), (60, 32, 32), (180,), (60,))

## 1.3 Image Matching

In [43]:
# ( imageA, imageB, method=cc|conv|ssd, normalize=y|n)
def matchingImages (a, b, method="cc", normalize='n'):
    
    result, np_result = 0, 0
    # define thresholds
    cc_threshold = 16260381 # 16250606
    conv_threshold = 16251165 # 16240502
    ssd_threshold = 475521 # 222979
    # initializations
    match = 0
    std = 1
    a_zero_mean = a.copy()
    b_zero_mean = b.copy()
    
    if normalize == 'y':
        std = (np.sum((b-np.mean(b))**2) * np.sum((a-np.mean(a))**2))** 0.5
        a_zero_mean = a - np.mean(a)
        b_zero_mean = b - np.mean(b)
        cc_threshold = 0.051464586812097785 # 0.042635353182592345
        conv_threshold = -3536 # -0.064 
        ssd_threshold = 1.9 # 1.93 
    
    a_zero_mean = a_zero_mean.flatten()
    b_zero_mean = b_zero_mean.flatten()
    a = a.flatten()
    b = b.flatten()
    
    if method == "cc":
        sum = 0
        for element_a, element_b in zip(a_zero_mean, b_zero_mean):
            sum = sum + element_a * element_b
        result = sum
        result = result / std
        # np_result = np.correlate(a_zero_mean,b_zero_mean)
        
        
        if result >= cc_threshold:
            match = 1
            
    
    if method == "conv":
        b_flipped = np.flip(b)
        sum = 0
        for element_a, element_b in zip(a_zero_mean, b_flipped):
            sum = sum + element_a * element_b
        result = sum
        # result = result / std
        # np_result= np.convolve(a_zero_mean,b)
        
        if result >= conv_threshold:
            match = 1
        
    if method == "ssd":
        sum = 0
        for element_a, element_b in zip(a_zero_mean, b_zero_mean):
            sum = sum + ((element_a - element_b)) ** 2
        result = sum
        result = result / std
        # np_result = np.sum((a - b)**2)
        
        if result <= ssd_threshold:
            match = 1
    return result, np_result, match

### Get the average of CC, Conv, SSD to be able to set the thresholds properly

### No normalization CC

In [30]:
big_sum = 0
i, j = 0, 0
# iter = 0
for i in [0,30,60,90,120,150]:
    sum = 0
    for j in range(i, i+29):
        result, _, match = matchingImages(X_train[j],X_train[j+1])
        sum += result
        # iter +=1
    print(sum/29)
    big_sum += sum/29
#print(iter)
print("___________")
print("Avergae: " ,big_sum/6)

# TEST

result, _, match = matchingImages(X_train[1],X_train[8])
print("Cross Correlation result:",result,"  || match: ", match)
result, _, match = matchingImages(X_train[1],X_train[100])
print("Cross Correlation result:",result,"  || match: ", match)
result, _, match = matchingImages(X_train[1],X_train[70])
print("Cross Correlation result:",result,"  || match: ", match)

16251497.379310345
16258055.603448275
16264102.323275862
16260259.36637931
16265953.534482758
16262420.85775862
___________
Avergae:  16260381.510775864
Cross Correlation result: 16252144.25   || match:  0
Cross Correlation result: 16239722.8125   || match:  0
Cross Correlation result: 16238906.9375   || match:  0


### No normalization Conv

In [31]:
big_sum = 0
i, j = 0, 0
for i in [0,30,60,90,120,150]:
    sum = 0
    for j in range(i, i+29):
        result, _, match = matchingImages(X_train[j],X_train[j+1], method = "conv")
        sum += result
    print(sum/29)
    big_sum += sum/29
print("___________")
print("Avergae: ", big_sum/6)

# TEST

result, _, match = matchingImages(X_train[1],X_train[150] , method = "conv")
print("Convolution result:",result,  "  || match: ", match)
result, _, match = matchingImages(X_train[119],X_train[120] , method = "conv")
print("Convolution result:",result, "  || match: ", match)
result, _, match = matchingImages(X_train[1],X_train[50] , method = "conv")
print("Convolution result:",result, "  || match: ", match)

16237012.978448275
16247005.898706896
16259012.76724138
16254975.484913792
16255743.459051725
16253244.217672413
___________
Avergae:  16251165.801005745
Convolution result: 16241995.8125   || match:  0
Convolution result: 16254816.75   || match:  1
Convolution result: 16250798.5625   || match:  0


### No normalization SSD

In [36]:
big_sum = 0
i, j = 0, 0
for i in [0,30,60,90,120,150]:
    sum = 0
    for j in range(i, i+29):
        result, _, match = matchingImages(X_train[j],X_train[j+1], method = "ssd")
        sum += result
    print(sum/29)
    big_sum += sum/29
print("___________")
print("Avergae: " , big_sum/6)

# TEST

result, _, match = matchingImages(X_train[1],X_train[15] , method = "ssd")
print("Sum of square differences result:",result, "  || match: ", match)
result, _, match = matchingImages(X_train[179],X_train[100] , method = "ssd")
print("Sum of square differences result:",result,"  || match: ", match)
result, _, match = matchingImages(X_train[30],X_train[150] , method = "ssd")
print("Sum of square differences result:",result, "  || match: ", match)

197076.68318965516
547059.504310345
839294.1422413794
487560.7521551724
78891.0086206896
703244.0969827586
___________
Avergae:  475521.03125
Sum of square differences result: 175151.0625   || match:  1
Sum of square differences result: 548936.75   || match:  0
Sum of square differences result: 684113.9999999999   || match:  0


### Normalized CC

In [39]:
big_sum = 0
i, j = 0, 0
for i in [0,30,60,90,120,150]:
    sum = 0
    for j in range(i, i+29):
        result, _, match = matchingImages(X_train[j],X_train[j+1], method = "cc", normalize = 'y')
        sum += result
    print(sum/29)
    big_sum += sum/29
print("___________")
print("Avergae: ", big_sum/6)

# TEST

result, _, match = matchingImages(X_train[1],X_train[7] , normalize = 'y')
print("Normalized Cross Correlation result:",result, "  || match: ", match)
result, _, match = matchingImages(X_train[1],X_train[120] , normalize = 'y')
print("Normalized Cross Correlation result:",result, "  || match: ", match)
result, _, match = matchingImages(X_train[1],X_train[70] , normalize = 'y')
print("Normalized Cross Correlation result:",result, "  || match: ", match)

0.05786909942394834
0.0019503285986316923
0.016636571575031508
0.010289518398571474
0.20585243593270655
0.016189566943697175
___________
Avergae:  0.051464586812097785
Normalized Cross Correlation result: 0.0772027002787194   || match:  1
Normalized Cross Correlation result: -0.0954746683498355   || match:  0
Normalized Cross Correlation result: -0.04444502948682246   || match:  0


### Normalized Conv

In [50]:
big_sum = 0
i, j = 0, 0
for i in [0,30,60,90,120,150]:
    sum = 0
    for j in range(i, i+29):
        result, _, match = matchingImages(X_train[j],X_train[j+1], method = "conv", normalize = 'y')
        sum += result
    print(sum/29)
    big_sum += sum/29
print("___________")
print("Avergae: ", big_sum/6)

# TEST

result, _, match = matchingImages(X_train[179],X_train[155] , method = "conv", normalize = 'y')
print("Normalized Convolution result:",result,"  || match: ", match)
result, _, match = matchingImages(X_train[1],X_train[2] , method = "conv", normalize = 'y')
print("Normalized Convolution result:",result,"  || match: ", match)
result, _, match = matchingImages(X_train[1],X_train[150] , method = "conv", normalize = 'y')
print("Normalized Convolution result:",result,"  || match: ", match)

-7816.1331871295
-10022.302492338973
2366.166840651933
-2573.0706850249558
46.709718901444454
-3222.5800718109235
___________
Avergae:  -3536.868312791829
Normalized Convolution result: 23274.510742187864   || match:  1
Normalized Convolution result: -14092.34649658195   || match:  0
Normalized Convolution result: -7782.621398925584   || match:  0


### Normalized SSD

In [51]:
big_sum = 0
i, j = 0, 0
for i in [0,30,60,90,120,150]:
    sum = 0
    for j in range(i, i+29):
        result, _, match = matchingImages(X_train[j],X_train[j+1], method = "ssd", normalize = 'y')
        sum += result
    print(sum/29)
    big_sum += sum/29
print("___________")
print("Avergae: ", big_sum/6)

# TEST

result, _, match = matchingImages(X_train[1],X_train[15] , method = "ssd", normalize = 'y')
print("Normalized Sum of square differences result:",result,"  || match:", match)
result, _, match = matchingImages(X_train[1],X_train[100] , method = "ssd", normalize = 'y')
print("Normalized Sum of square differences result:",result, "  || match:", match)
result, _, match = matchingImages(X_train[1],X_train[50] , method = "ssd", normalize = 'y')
print("Normalized Sum of square differences result:",result,"  || match:", match)

1.8924298839827622
2.0000764517046914
1.9681731166518655
1.984934889317656
1.5908062206131424
1.9692285643837524
___________
Avergae:  1.9009415211089784
Normalized Sum of square differences result: 1.688754246324758   || match: 1
Normalized Sum of square differences result: 2.2410422985098233   || match: 0
Normalized Sum of square differences result: 2.1697392859568807   || match: 0


## Preparing Train and Validation Set
#### take 10 images of each category for validation and the rest 20 for train

In [52]:
X_val = []
y_val = []
i, j = 0, 0
for i in [0,30,60,90,120,150]:
    for j in range(i, i+10):
        X_val.append(X_train[j])
        y_val.append(y_train[j])
        
X_val = np.array(X_val)
y_val = np.array(y_val)

X_val.shape, y_val.shape

((60, 32, 32), (60,))

In [53]:
y_val

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [54]:
# new train set
X_train_reduced = []
y_train_reduced = []
i, j = 0, 0
for i in [0,30,60,90,120,150]:
    for j in range(i+10,i+30):
        X_train_reduced.append(X_train[j])
        y_train_reduced.append(y_train[j])
X_train_reduced = np.array(X_train_reduced)
y_train_reduced = np.array(y_train_reduced)
print(X_train_reduced.shape, y_train_reduced.shape)
print(y_train_reduced)

(120, 32, 32) (120,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]


In [55]:
# reshape these sets
X_train_reduced_flattened = X_train_reduced.reshape(X_train_reduced.shape[0], -1)
X_val_flattened = X_val.reshape(X_val.shape[0], -1)
X_train_reduced_flattened.shape , X_val_flattened.shape

((120, 1024), (60, 1024))

## Matching validation using validation set

In [56]:
def validate_matching_classifier(X, y, method = "cc", normalize = 'n'):
    i = 0
    j = 0
    positive_wins = 0
    negative_wins = 0
    positive_fails = 0
    negative_fails = 0
    no_iter = 0
    for i in range(X.shape[0]):
        for j in range (i+1,X.shape[0]):
            no_iter += 1
            result, np_result, match = matchingImages(X[i],X[j], method = method, normalize=normalize)
            # all the matching should be 1 except for 6 or 7 iterations
            if y[i] == y[j] :
                positive_wins += match
                positive_fails += (not match)
                    
            elif y[i] != y[j]:
                negative_wins += (not match)
                negative_fails += match
                
                
        
    print('TP: ', positive_wins)
    print('TN: ', negative_wins)
    print('FP: ', negative_fails)
    print('FN: ', positive_fails)
    print('Number of tests: ', no_iter)
    print("__________")
    
    return positive_wins, negative_wins, positive_fails, negative_fails

In [57]:
TP, TN, FP, FN = validate_matching_classifier(X_val_flattened, y_val, method = "cc", normalize = 'n')

TP:  132
TN:  944
FP:  556
FN:  138
Number of tests:  1770
__________


In [61]:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
Recall = TP / (TP + FN)
Precision =  TP / (TP + FP)
F1 = (2 * Precision * Recall) / (Precision + Recall)
print ( "Accuracy: ", Accuracy,
       "\nRecall: ", Recall,
       "\nPrecesion: ", Precision,
       "\nF1 Score: ", F1)

Accuracy:  0.607909604519774 
Recall:  0.19186046511627908 
Precesion:  0.4888888888888889 
F1 Score:  0.2755741127348643


## Prepare concatenated dataset

### Train Set

In [63]:
X_train_reduced_conc = []
y_train_reduced_conc = []
i, j, k = 0, 0, 0
for i in range(X_train_reduced.shape[0]):
    for j in range(i+1, X_train_reduced.shape[0]):
        #if i != j:
        X_train_reduced_conc.append(np.dstack([X_train_reduced[i], X_train_reduced[j]]))
        if y_train_reduced[i] == y_train_reduced [j]:
            y_train_reduced_conc.append(1)
        else:
            y_train_reduced_conc.append(0)
X_train_reduced_conc = np.array(X_train_reduced_conc)
y_train_reduced_conc = np.array(y_train_reduced_conc)
X_train_reduced.shape , y_train_reduced.shape, X_train_reduced_conc.shape , y_train_reduced_conc.shape            

((120, 32, 32), (120,), (7140, 32, 32, 2), (7140,))

### Validation Set

In [64]:
X_val_conc = []
y_val_conc = []
i, j, k = 0, 0, 0
for i in range(X_val.shape[0]):
    for j in range(i+1, X_val.shape[0]):
        #if i != j:
        X_val_conc.append(np.dstack([X_val[i], X_val[j]]))
        if y_val[i] == y_val[j]:
            y_val_conc.append(1)
        else:
            y_val_conc.append(0)
X_val_conc = np.array(X_val_conc)
y_val_conc = np.array(y_val_conc)
X_val.shape , y_val.shape, X_val_conc.shape , y_val_conc.shape          

((60, 32, 32), (60,), (1770, 32, 32, 2), (1770,))

In [65]:
# flatten them to use for the MLP
X_train_reduced_conc_flattened = X_train_reduced_conc.reshape(X_train_reduced_conc.shape[0], -1)
X_val_conc_flattened = X_val_conc.reshape(X_val_conc.shape[0], -1)
X_train_reduced_conc_flattened.shape , X_val_conc_flattened.shape

((7140, 2048), (1770, 2048))

## Train on MLP and test using Validation Set

In [66]:
from sklearn.neural_network import MLPClassifier
clf_base = MLPClassifier(random_state=1, max_iter=300).fit(X_train_reduced_conc_flattened, y_train_reduced_conc)
print(clf_base.score(X_train_reduced_conc_flattened, y_train_reduced_conc) ,
      clf_base.score(X_val_conc_flattened, y_val_conc))

0.8984593837535014 0.6412429378531074


In [67]:
from sklearn.metrics import classification_report
y_pred = clf_base.predict(X_val_conc_flattened)
print(classification_report(y_val_conc, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.70      0.77      1500
           1       0.16      0.31      0.21       270

    accuracy                           0.64      1770
   macro avg       0.50      0.51      0.49      1770
weighted avg       0.74      0.64      0.68      1770



In [68]:
unique, counts = np.unique(y_train_reduced_conc, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 6000],
       [   1, 1140]])

#### We notice the great unbalance in the data, maybe if we fix this the classifier will be better

## Re-sample Train Set
#### (Under sampling)

In [69]:
from imblearn.under_sampling import NearMiss
from imblearn import under_sampling
from imblearn.over_sampling import RandomOverSampler
# define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3)
# transform the dataset
X_resampled, y_resampled = undersample.fit_resample(X_train_reduced_conc_flattened, y_train_reduced_conc)
X_resampled.shape, y_resampled.shape

((2280, 2048), (2280,))

In [70]:
unique, counts = np.unique(y_resampled, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 1140],
       [   1, 1140]])

### Train our MLP on this new Resampled Train Set

In [74]:
clf_resampled = MLPClassifier(random_state=1, max_iter=300).fit(X_resampled,y_resampled)
print(clf_resampled.score(X_resampled, y_resampled) ,
      clf_resampled.score(X_val_conc_flattened, y_val_conc))

0.7649122807017544 0.5830508474576271


In [75]:
y_pred = clf_resampled.predict(X_val_conc_flattened)
print(classification_report(y_val_conc, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.62      0.72      1500
           1       0.15      0.36      0.21       270

    accuracy                           0.58      1770
   macro avg       0.50      0.49      0.46      1770
weighted avg       0.74      0.58      0.64      1770



- The performance either did not change or became worse, this could be due that the undersampling resulted in less features, which resulted in the model capturing less details and patterns. A solution to this could be to try oversampling instead. Anyway for now let's stick with our clf_base model.

## Classification Comparison
 
 - As we can see the accuracy of the 1st classifier is almost 60%, F1_score = 27% . While the accuracy of MLP is 64% and one F1 is much higher up to 77% for class0. This is an indication of the generlization ability of MLP model. 
 - Regarding the training effort, of course the MLP takes much longer, however, it's more robust. Because hardcoding one threshold for each matching method in the 1st classifier is brone to error and can be actually useless in a real world example where we can not rely on conditional ifs. 
 - The prediction speed of the 1st classifier is higher, because all it does is go through an if condition, while the MLP goes through the multi layers and all of their accompyning calculations in order to reach a prediction.
 - In general, unless the application is trivial matching application and unless the speed is not an issue, I'd recommend MLP for its generalization and robustness.

## 1.6 Feature Engineering
### Apply filters and features extraction on data before resampling

In [76]:
X_train_reduced_conc.shape, X_train_reduced_conc.shape

((7140, 32, 32, 2), (7140, 32, 32, 2))

### Apply Sobel edge detection to help identify the texture

In [77]:
from skimage.filters import sobel
X_train_reduced_conc_sobeled = []
for image in X_train_reduced_conc:
    sobeled = sobel(image)
    X_train_reduced_conc_sobeled.append(sobeled)
    

In [78]:
X_train_reduced_conc_sobeled = np.array(X_train_reduced_conc_sobeled)
X_train_reduced_conc_sobeled.shape

(7140, 32, 32, 2)

In [79]:
X_train_reduced_conc_sobeled_flattened = X_train_reduced_conc_sobeled.reshape(
    X_train_reduced_conc_sobeled.shape[0], -1)
X_train_reduced_conc_sobeled.shape, X_train_reduced_conc_sobeled_flattened.shape

((7140, 32, 32, 2), (7140, 2048))

### Apply PCA to reduce dimensionality to 32

In [80]:
from sklearn.decomposition import PCA
pca = PCA(n_components=32)
X_train_reduced_conc_sobeled_flattened_pca = pca.fit_transform(X_train_reduced_conc_sobeled_flattened)
X_train_reduced_conc_sobeled_flattened_pca.shape

(7140, 32)

#### Now apply the same things to the validation set

In [81]:
X_val_conc_sobeled = []
for image in X_val_conc:
    sobeled = sobel(image)
    X_val_conc_sobeled.append(sobeled)
X_val_conc_sobeled = np.array(X_val_conc_sobeled)
X_val_conc_sobeled.shape

(1770, 32, 32, 2)

In [82]:
X_val_conc_sobeled_flattened = X_val_conc_sobeled.reshape(X_val_conc_sobeled.shape[0], -1)
X_val_conc_sobeled.shape, X_val_conc_sobeled_flattened.shape

((1770, 32, 32, 2), (1770, 2048))

In [83]:
X_val_conc_sobeled_flattened_pca = pca.transform(X_val_conc_sobeled_flattened)
X_val_conc_sobeled_flattened_pca.shape

(1770, 32)

### Apply same feature engineering on data after resampling

In [84]:
X_resampled.shape, y_resampled.shape

((2280, 2048), (2280,))

In [85]:
X_train_conc_sobeled_resampled = []
for image in X_resampled:
    sobeled = sobel(image)
    X_train_conc_sobeled_resampled.append(sobeled)
X_train_conc_sobeled_resampled = np.array(X_train_conc_sobeled_resampled)
X_train_conc_sobeled_resampled.shape

(2280, 2048)

In [86]:
pca_resampled = PCA(n_components=32)
X_train_conc_sobeled_resampled_pca = pca_resampled.fit_transform(X_train_conc_sobeled_resampled)


#### Now only apply the PCA to the Validation Set (The Sobel has been applied before and it's not different)

In [88]:
X_val_resampled_pca = pca_resampled.transform(X_val_conc_sobeled_flattened)
X_val_conc_sobeled_flattened.shape , X_val_resampled_pca.shape

((1770, 2048), (1770, 32))

### Apply MLP on no resample feature engineered


In [89]:
from sklearn.utils import shuffle

X_train_reduced_conc_sobeled_flattened_pca_shuffled, y_train_reduced_conc_shuffled= shuffle(X_train_reduced_conc_sobeled_flattened_pca, y_train_reduced_conc, random_state = 4)

In [90]:
clf_ = MLPClassifier(random_state = 4, max_iter=500).fit(X_train_reduced_conc_sobeled_flattened_pca_shuffled, y_train_reduced_conc_shuffled)

print(clf_.score(X_train_reduced_conc_sobeled_flattened_pca_shuffled, y_train_reduced_conc_shuffled) ,
      clf_.score(X_val_conc_sobeled_flattened_pca, y_val_conc))

0.999859943977591 0.8903954802259887


In [91]:
y_pred = clf_.predict(X_val_conc_sobeled_flattened_pca)
print(classification_report(y_val_conc, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.91      0.93      1500
           1       0.61      0.79      0.69       270

    accuracy                           0.89      1770
   macro avg       0.78      0.85      0.81      1770
weighted avg       0.91      0.89      0.90      1770



#### Much much better accuracy !! 
#### Now let's try this on the resampled feature engineered

### Apply MLP on Resample feature engineered


In [92]:
X_train_conc_sobeled_resampled_pca_shuffled, y_resampled_shuffled= shuffle(X_train_conc_sobeled_resampled_pca, y_resampled, random_state = 4)

In [95]:
clf = MLPClassifier(random_state = 4, max_iter=500).fit(X_train_conc_sobeled_resampled_pca_shuffled, y_resampled_shuffled)

print(clf.score(X_train_conc_sobeled_resampled_pca_shuffled, y_resampled_shuffled) ,
      clf.score(X_val_resampled_pca, y_val_conc))

0.9956140350877193 0.22090395480225988


In [96]:
y_pred = clf.predict(X_val_resampled_pca)
print(classification_report(y_val_conc, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.09      0.17      1500
           1       0.16      0.94      0.27       270

    accuracy                           0.22      1770
   macro avg       0.52      0.51      0.22      1770
weighted avg       0.78      0.22      0.18      1770



#### Again, undersampling resulting in very bad results. 

## 1.7 Discussion on Feature Engineering

### First prepare the test data to suit our pipeline

In [97]:
X_test.shape

(60, 32, 32)

In [98]:
X_test_conc = []
y_test_conc = []
i, j, k = 0, 0, 0
for i in range(X_test.shape[0]):
    for j in range(i+1, X_test.shape[0]):
        X_test_conc.append(np.dstack([X_test[i], X_test[j]]))
        if y_test[i] == y_test [j]:
            y_test_conc.append(1)
        else:
            y_test_conc.append(0)
            

In [99]:
X_test_conc = np.array(X_test_conc)
y_test_conc = np.array(y_test_conc)
X_test_conc.shape , y_test_conc.shape

((1770, 32, 32, 2), (1770,))

In [100]:
X_test_conc_sobeled = []
for image in X_test_conc:
    # sharpened = unsharp_mask(image, radius=20, amount=1)
    sobeled = sobel(image)
    X_test_conc_sobeled.append(sobeled)
X_test_conc_sobeled = np.array(X_test_conc_sobeled)
X_test_conc_sobeled.shape

(1770, 32, 32, 2)

In [101]:
X_test_conc_sobeled_flattened = X_test_conc_sobeled.reshape(X_test_conc_sobeled.shape[0], -1)
X_test_conc_sobeled_flattened.shape

(1770, 2048)

In [102]:
X_test_conc_sobeled_flattened_pca = pca.transform(X_test_conc_sobeled_flattened)
X_test_conc_sobeled_flattened_pca.shape

(1770, 32)

### Test noResample_FeatureEngineered Classifier (our best one so far) on TEST DATA

In [103]:
y_pred = clf_.predict(X_test_conc_sobeled_flattened_pca)
print(classification_report(y_test_conc, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.91      0.94      1500
           1       0.62      0.79      0.70       270

    accuracy                           0.89      1770
   macro avg       0.79      0.85      0.82      1770
weighted avg       0.91      0.89      0.90      1770



### Test our first base classifier (before feature engineering) on TEST DATA

In [106]:
y_pred = clf_base.predict(X_test_conc.reshape(X_test_conc.shape[0],-1))
print(classification_report(y_test_conc, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.61      0.71      1500
           1       0.15      0.37      0.21       270

    accuracy                           0.57      1770
   macro avg       0.49      0.49      0.46      1770
weighted avg       0.74      0.57      0.63      1770



## Discussion
- By the looking at the last two classifier experiments, we can see the great improvement that the feature engineering (the Sobel filter only) and the feature extraction by dimensionality reduction (PCA) have done to the model.
- The improvement is not just in the overall accuracy, it's also in the precision, recall scores which reflects the model's performance on rare cases and its generelization ability.
- Regarding my choice of the filter. Since our data is texture images, I figured that the edges would be a good representation of each image. Given that the images' colors and all the other details are of no importance, only the edges and their relationship to eachother matters. Because that's what defines a texture.
- Regarding the PCA choice, it's the most famous and most efficient dimensionality reduction method I know, and given that we have had so many features that we wanted to get rid of, it seemed like a reasonable approach and so it was.
- A side note on the accuracy metrics of our champion model: The reason why class0 scores are higher than class1 is because class0 exists more in the train set. a solution to this could be to oversample the rarest class instead of undersample because as we saw, undersampling did not do very well, the model needed more data to be able to capture the details (it underfitted). Another solution could be to make the model more complex as well.
